# Dealing with Time

The [MIMIC III documentation](https://mimic.physionet.org/mimicdata/time/) describes all of the time fields. Some key points are:  
"Time in the database is stored with one of two suffixes: TIME and DATE. If a column has TIME as the suffix, e.g. CHARTTIME, then the data resolution is down to the minute. If the column has DATE as the suffix, e.g. CHARTDATE, then the data resolution is down to the day."

In [1]:
import pandas as pd

### Obtain a list of all timestamp fields in the MIMIC-III data  
- Copy table of all data columns from https://mit-lcp.github.io/mimic-schema-spy/columns.byTable.html into a spreadsheet
- Filter the "Type" column on timestamp and delete all other rows
- Note that all timestamp columns have a size of 22 characters, suggesting that they likely have a uniform format across the database
- Keep only the "Table" and "Column" columns and save to CSV

In [2]:
time_fields = pd.read_csv('MIMIC-III_timestamp_fields.csv')

In [3]:
time_fields.loc[:,'Table'] = time_fields.loc[:,'Table'].str.title()
time_fields.loc[:,'Column'] = time_fields.loc[:,'Column'].str.lower()
time_fields

,Table,Column
0,Admissions,admittime
1,Admissions,deathtime
2,Admissions,dischtime
3,Admissions,edouttime
4,Admissions,edregtime
...,...,...
69,Procedureevents_Mv,starttime
70,Procedureevents_Mv,storetime
71,Services,transfertime
72,Transfers,intime


### Initialize a connection to the neo4j database.

In [4]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j','Gr@ph3HR'))
session=driver.session()

### MERGE all timestamps as relationships into the time tree

In [5]:
# Create the root node of the time tree
query = 'MERGE (t:Timetree {name:"Time Tree"})'
session.run(query)

In [6]:
# Create all the year, month, and day nodes with relationships

# Column from MIMIC = relationship name
# Properties of relationship: timestamp
# Each node contains the datetime informationa for all higher nodes in the tree (example: month nodes contain

count = 0
for index, row in time_fields.iterrows():
    Label = row[0]
    prop = row[1]
    query = '''
"MATCH (n:{Label})
WHERE n.{prop} =~ '[0-9]{{4}}-[0-9]{{2}}-[0-9]{{2}} [0-9]{{2}}:[0-9]{{2}}:[0-9]{{2}}'
WITH apoc.date.parse(n.{prop}, 'ms', 'yyyy-MM-dd HH:mm:ss', 'America/New York') AS ms, n
WITH datetime({{epochmillis: ms}}).year AS yr, datetime({{epochmillis: ms}}).month AS mo, datetime({{epochmillis: ms}}).day AS dt,
ms, n
RETURN yr, mo, dt, ms, n",
"MATCH (t:Timetree {{name:'Time Tree'}})
MERGE (t)<-[:OF]-(y:Year {{year:yr}})
MERGE (y)<-[:OF]-(m:Month {{year:yr, month:mo}})
MERGE (m)<-[:OF]-(d:Day {{year:yr, month:mo, day:dt}})
MERGE (d)<-[:{prop} {{{prop}:datetime({{epochmillis: ms}})}}]-(n)"'''.format(Label=Label, prop=prop)
    count += 1
    print(Label, prop, count)
    command = 'CALL apoc.periodic.iterate('+query+', {batchSize:1000, parallel: true, iterateList:true})'
    session.run(command)


# Print a test query
# print(command)

Admissions admittime 1
Admissions deathtime 2
Admissions dischtime 3
Admissions edouttime 4
Admissions edregtime 5
Callout acknowledgetime 6
Callout createtime 7
Callout currentreservationtime 8
Callout firstreservationtime 9
Callout outcometime 10
Callout updatetime 11
Chartevents charttime 12
Chartevents storetime 13
Chartevents_1 charttime 14
Chartevents_1 storetime 15
Chartevents_10 charttime 16
Chartevents_10 storetime 17
Chartevents_11 charttime 18
Chartevents_11 storetime 19
Chartevents_12 charttime 20
Chartevents_12 storetime 21
Chartevents_13 charttime 22
Chartevents_13 storetime 23
Chartevents_14 charttime 24
Chartevents_14 storetime 25
Chartevents_2 charttime 26
Chartevents_2 storetime 27
Chartevents_3 charttime 28
Chartevents_3 storetime 29
Chartevents_4 charttime 30
Chartevents_4 storetime 31
Chartevents_5 charttime 32
Chartevents_5 storetime 33
Chartevents_6 charttime 34
Chartevents_6 storetime 35
Chartevents_7 charttime 36
Chartevents_7 storetime 37
Chartevents_8 chartti

### Close the connection to the neo4j database

In [8]:
session.close()